In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import smtplib
from email.mime.text import MIMEText
import os

def get_genre():
    print("Choose your favorite genre:")
    print("1. Detectives")
    print("2. Fantasy")
    print("3. Science Fiction")
    choice = input("Enter the number of your choice (1/2/3): ").strip()
    
    genres = {
        '1': 'detectives',
        '2': 'fantasy',
        '3': 'science-fiction'
    }
    
    while choice not in genres:
        print("Invalid choice. Please try again.")
        choice = input("Enter the number of your choice (1/2/3): ").strip()
    
    return genres[choice]

def scrape_amazon(genre, limit=10):
    url = f"https://www.amazon.com/best-sellers-books-{genre}/zgbs/books"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data from Amazon: {e}")
        return []
    
    soup = BeautifulSoup(response.content, 'html.parser')
    books = []
    
    for item in soup.select('.zg-item-immersion'):
        title = item.select_one('.p13n-sc-truncate')
        if title:
            books.append(title.get_text(strip=True))
        if len(books) == limit:
            break
    
    return books

def validate_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)

def send_email(books, genre, email):
    subject = f"Top 10 {genre.capitalize()} Books"
    message = f"Here are the top 10 best-selling {genre} books on Amazon:\n\n"
    message += "\n".join([f"{i+1}. {book}" for i, book in enumerate(books)])

    sender_email = os.getenv("SENDER_EMAIL")  # Use environment variable
    sender_password = os.getenv("SENDER_PASSWORD")  # Use environment variable

    msg = MIMEText(message)
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = email

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, [email], msg.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

def main():
    genre = get_genre()
    print("\nFetching the top 10 books...")
    books = scrape_amazon(genre)
    
    if not books:
        print("No books found. Please try again later.")
        return
    
    print("\nTop 10 Books:")
    for i, book in enumerate(books, 1):
        print(f"{i}. {book}")
    
    send_choice = input("\nWould you like to receive this list by email? (yes/no): ").strip().lower()
    if send_choice == 'yes':
        while True:
            email = input("Enter your email address: ").strip()
            if validate_email(email):
                break
            print("Invalid email address. Please try again.")
        
        send_email(books, genre, email)

if __name__ == "__main__":
    main()